***
### [torch.autograd](https://pytorch.org/docs/stable/autograd.html#)
autograd主要提供自动微分的类和函数，需要在创建tensor时指定requires_grad=True，并且只支持float相关类型和复数tensor类型。

PyTorch提供backward()和torch.autograd.grad()两种求梯度的方法。backward()会将梯度填充到叶子节点的.grad字段，而torch.autograd.grad()直接返回梯度结果。
|||
|---|---|
|backward|用于启动反向传播，计算图中所有叶子节点的梯度|
|grad|计算输出相对输入的梯度|

***
```python
torch.autograd.grad(outputs, inputs, grad_outputs=None, retain_graph=None, create_graph=False, only_inputs=True, allow_unused=None, is_grads_batched=False, materialize_grads=False)
```
**Parameters**
* outputs   - 输出张量
* inputs    - 输入张量
* grad_outputs  - 如果outputs不是标量，需要使用此参数
* retain_graoh  - 保留计算图
* create_graph  - 创建计算图，用于高阶导数计算
* allow_unused  - 允许输入变量不进入计算
* is_grads_batched  - 如果为True，grad_output中每个张量的第一个维度将被解释为批量维度
* materialize_grads - 如果为True，输入的张量使用0进行填充
**Return Type**
* Tuple[Tensor,] - 输出梯度元组

In [6]:
import torch
from torch import autograd

x = torch.rand(3, 4)
x.requires_grad_()

# y为x中元素和，为标量
y = torch.sum(x)
grad1 = autograd.grad(outputs=y, inputs=x)
print(grad1)

# y为向量
y = x[:,0] + x[:,1]
grad2 = autograd.grad(outputs=y, inputs=x, grad_outputs=torch.ones_like(y))
print(grad2)

# 二阶导数
y = x ** 2
grad3 = autograd.grad(outputs=y, inputs=x, grad_outputs=torch.ones_like(y), create_graph=True)
grad4 = autograd.grad(outputs=grad3, inputs=x, grad_outputs=torch.ones_like(y))
print(grad4)

(tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]]),)
(tensor([[1., 1., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 0., 0.]]),)
(tensor([[2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.]]),)


***
```python
torch.autograd.backward(tensors, grad_tensors=None, retain_graph=None, create_graph=False, grad_variables=None, inputs=None)
```
**Parameters**
* tensors       - 指定要求梯度的张量
* grad_tensors  - 通常是对应张量的每个元素的梯度
* retain_graph  - 保留计算图
* create_graph  - 创建计算图，用于高阶导数计算
* inputs        - 指定计算的张量，若不指定则为所有叶子张量

In [7]:
import torch
from torch import autograd

x = torch.tensor(1.0, requires_grad=True)
y = torch.tensor(2.0, requires_grad=True)
z = x ** 2 + y
z.backward()
print(z, x.grad, y.grad)

tensor(3., grad_fn=<AddBackward0>) tensor(2.) tensor(1.)


***
### [torch.autograd.Function](https://pytorch.org/docs/stable/autograd.html#function)
包含了一系列用于定义自定义操作的函数，这些操作可以在反向传播时自动计算梯度。Function中的操作类似于计算图中的边。
|||
|---|---|
|Function.forward|自定义前向传播逻辑|
|Function.backward|自定义反向传播逻辑|
|Function.jvp|自定义jvp过程|
|Function.vmap|自定义torch.vmap()的求导过程|

In [9]:
import torch

# ex的自动求导
class Exp(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i.exp()
        ctx.save_for_backward(result)
        return result
    
    @staticmethod
    def backward(ctx, grad_output):
        result, = ctx.saved_tensors
        return grad_output * result
    
input = torch.tensor(2.0, requires_grad=True)
output = Exp.apply(input)
print(output)
print(input.grad)

tensor(7.3891, grad_fn=<ExpBackward>)
tensor(7.3891)


***
### [torch.autograd.functional](https://pytorch.org/docs/stable/autograd.html#functional-higher-level-api)
torch.autograd.functional这个为autograd提供了更高级api，并且可以计算jacobians核hessians等。
|||
|---|---|
|functional.jacobian|计算给定函数的雅可比|
|functional.hessian|计算给定函数的黑森|
|functional.vjp|计算给定点的向量和雅可比的点积|
|functional.jvp|计算给定点的雅可比和向量的点积|
|functional.vhp|计算给定点的向量和黑森的点积|
|functional.hvp|计算给定点的黑森和向量的点积|

***
### 其它
* torch.autograd.gradcheck 用于检查数值梯度与自动微分得到的梯度是否一致，这是确保正确性的一个有用工具。
* torch.autograd.detect_anomaly 在自动求导时检测错误产生路径，有助于调试。
* torch.autograd.grad_mode 允许用户设置是否需要梯度，例如在模型评估时通常不需要计算梯度。